### Writing a single band raster CreateCopy method

>There are two general techniques for creating files, using CreateCopy() and Create(). The CreateCopy method involes calling the CreateCopy() method on the format driver, and passing in a source dataset that should be copied. The Create method involves calling the Create() method on the driver, and then explicitly writing all the metadata, and raster data with separate calls.  

>https://gdal.org/drivers/raster/index.html

In [ ]:
fileformat='GTiff'

In [ ]:
driver=gdal.GetDriverByName(fileformat)

In [ ]:
print(driver.ShortName)

In [ ]:
print(driver.LongName)

In [ ]:
metadata=driver.GetMetadata()

In [ ]:
metadata

### Using CreateCopy()

> The GDALDrviver::CreateCopy() method is fairly simply as most information is collected from the source dataset.

In [ ]:
scr_filename='give file name'

In [ ]:
dst_filename=r''
src_ds=gdal.open(src_file)

In [ ]:
driver=gdal.GetDriverByName('GTiff')

In [ ]:
dst_ds=driver.CreateCopy(dst_filename, src_ds)

In [ ]:
dst_ds

In [ ]:
data=src_ds.ReadAsArray()

In [ ]:
inv_data=np.invert(data)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(data,cmap='gray')

plt.subplot(1,2,2)
plt.imshow(inv_data, cmap='gray')

In [ ]:
out_dand=dst_ds.GetRasterBand(1)

In [ ]:
out_band.WriteArray(inv_data)

In [ ]:
out_band.FlushCache()

In [ ]:
del dst_ds

### Writing a single band raster Create Method

>For situations in which we are not just exporting an existing file to a new file, it is generally necessary to be use the GDALDriver::Create() method image size, number of bands and band type must be provided explicitly. 

>>A csv file containing daily gridded rainfall data set over india.

>>The first data in the record is at 6.5N & 66.5E, the second is at 6.5N & 66.75E, and so on

>>The second row 6.75N & 66.5E, the second is at 6.75N & 66.75E,

>>The last data record corresponds to 38.5N & 100.0E,

>>Latitude : 6.5N 38.5N.25

>>Longitude : 66.5E 100.0E.25

>>Rows and Column : 129x135

In [ ]:
src_txt_file='rainfall.txt'

In [ ]:
dst_filename='op_raster.tif'

In [ ]:
driver=gdal.GetDriverByName('GTiff')

In [ ]:
dst_ds=driver.Create(dst_filename,xsize=135, ysize=129, bands=1, etype=gdal.GDT_Float32)

In [ ]:
data=np.loadtxt(src_txt_file,delimiter=',',dtype=np.float32)

In [ ]:
data

In [ ]:
band=dst_ds.GetRasterBand(1)

### Adding Metadata with band

In [ ]:
from datetime import datetime

In [ ]:
band_meta={'Date of Creation': str(datetime.now()), 'scale_factor':'1', 'unit':'mm'}

In [ ]:
band_meta

In [ ]:
band.SetMetadata(band_meta)

### Adding NoData with band

In [ ]:
band.SetNoDataVaule(-999.00)

In [ ]:
band.WriteArray(data)

### Adding metadata to dataset

In [ ]:
ds_meta={'Data of Creation':str(datetime.now()),'organisation':'1','processing level':'L3'}

In [ ]:
dst_ds.SetMetaData(ds_meta)

In [ ]:
dst_ds.FlushCache()
dst_ds=None

In [ ]:
ds=gdal.Open(dst_filename)

In [ ]:
ds.GetMetadata()

In [ ]:
band=ds.GetRasterBand(1)

In [ ]:
band.GetMetadata()

In [ ]:
band.GetNoDataValue()

In [ ]:
ds=None

### Writing a Georefenced Raster

In [ ]:
from osgeo import osr

In [ ]:
crs = osr.SpatialReference()

In [ ]:
crs

In [ ]:
crs.SetWellKnownGeoCS('WGS84')

In [ ]:
crs.ExportToWkt()

In [ ]:
crs.ExportToProj4()

In [ ]:
trf=[66.5,.25,0.0,6.5,0.0,.25]

In [ ]:
src_txt_file='rainfall.txt'
dst_filename='image address'

In [ ]:
driver=gdal.GetDriverByName('HFA')

In [ ]:
dst_ds=driver.Create(dst_filename,xsize=135,ysize=129,bands=1,eType=gdal.GDT_float32)

In [ ]:
dst_ds.SetProjection(crs.ExportToWkt())

In [ ]:
dst_ds.SetGeoTransform(trf)

In [ ]:
band=dst_ds.GetRasterBand(1)

In [ ]:
band.SetNoDataValue(-999)

In [ ]:
data=np.loadtxt(src_txt_file,delimiter=',',dtype=np.int16)

In [ ]:
band.WriteArray(data)

In [ ]:
dst_ds.FlushCache()
dst_ds=None

### Read and confirm

In [ ]:
ds=gdal.Open(dst_filename)

In [ ]:
ds.GetProjection()

In [ ]:
ds.GetGeotransform()

In [ ]:
data=ds.ReadAsArray()

In [ ]:
plt.imshow(data,origin='lower')
plt.colorbar()

In [ ]:
no_data=ds.GetRasterBand(1).GetNoDataValue()

In [ ]:
no_data

In [ ]:
import numpy.ma as ma

In [ ]:
masked_data=ma.masked_equal(data, no_data)

In [ ]:
plt.imshow(masked_data, origin='lower')
plt.colorbar()

In [ ]:
plt.imshow(masked_data)
plt.colorbar()

In [ ]:
del ds

In [ ]:
os.chdir(r'')

### Stacking Raster Bands

In [ ]:
file_names=['file 1','file 2','file 3']

In [ ]:
img_data=list()
for file in file_names:
    ds=gdal.Open(file,gdal.GA_ReadOnly)
    data=ds.ReadAsArray()
    img_data.append(data)

In [ ]:
img_data

In [ ]:
fig=plt.figure(figsize=(20,25))
for i in range(1,4):
    plt.subplot(1,3,i)
    plt.imshow(img_data[i-1],cmap='gray')

In [ ]:
dst_file='RGB_IMG.tif'
height,width=img_data[0].shape

In [ ]:
height, width

In [ ]:
drv=gdal.GetDriverByName('GTiff')
dst_ds=drv.Create(dst_file, width, height, bands=3, eType=gdal.GDT_UInt16)

In [ ]:
dst_ds.SetProjection(ds.GetProjection())
dst_ds.SetGeoTransform(ds.GetGeoTransform())

In [ ]:
dst_ds.GetProjection()

In [ ]:
for i in range(1,4):
    band=dst_ds.GetRasterBand(i)
    band.WriteArray(img_data[i-1])
    #band.SetColorInterpretation(i+2)

In [ ]:
dst_ds.FlushCache()

In [ ]:
dst_ds=None

In [ ]:
ds=gdal.Open(dst_file)

In [ ]:
data=ds.ReadAsArray(buf_type=gdal.GDT_Byte)

In [ ]:
rgb_data=np.dstack(tuple(data))

In [ ]:
fig=plt.figure(figsize=(20,25))
plt.imshow(rgb_data)

In [ ]:
plt.close()

In [ ]:
del data,rgb_data,ds

### Reordering Bands of a Raster

In [ ]:
src_file='stacked_RGB_IMG.tif'

In [ ]:
ds=gdal.Translate('rgb_reorder.tif',src_file,bandList=[3,2,1])   # we can reorder the band list

In [ ]:
data=ds.ReadAsArray()

In [ ]:
rgb_data=np.dstack(tuple(data))

In [ ]:
fig=plt.figure(figsize=(20,25))
plt.imshow(rgb_data)

In [ ]:
plt.close()

### Subsetting raster using real-world coordinates

In [ ]:
os.chdir(r'')

In [ ]:
file_name='file name'

In [ ]:
ds=gdal.Open(file_name)

In [ ]:
gt=ds.GetGeoTansform()

In [ ]:
print('upper left corner : ',gdlApplyGeoTransform(gt,0,0))
print('upper right corner : ',gdalApplyGeotransform(gt,ds.RasterXSize,0))
print('lower left corner : ',gdalApplyGeotransform(gt,0,ds.RasterYSize))
print('lower right corner : ',gdalApplyGeotransform(gt,ds.RasterXSize,ds.RasterYSize))

In [ ]:
subset_extents=[78.15,78.20,29.75,29.80]

In [ ]:
inv_gt=gdal.InvGeoTransform(gt)

In [ ]:
offsets=gdal.ApplyGeoTransform(inv_gt,subset_extents[0],subset_extents[3])

In [ ]:
xoff,yoff=map(int,offsets)

In [ ]:
xoff,yoff

In [ ]:
offsets_2=gdal.ApplyGeoTransform(inv_gt,subset_extents[1],subset_extents[2])

In [ ]:
xoff_2,yoff_2=map(int,offsets_2)

In [ ]:
xoff_2,yoff_2

In [ ]:
rows=yoff_2-yoff

In [ ]:
colums=xoff_2-xoff

In [ ]:
print(rows, columns)

In [ ]:
data=ds.ReadAsArray(xoff,yoff,columns,rows)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(ds_ReadAsArray(),cmap='gray')

plt.subplot(1,2,2)
plt.imshow(data,cmap='gray')

In [ ]:
gt_subset=list(gt)

In [ ]:
gt_subset[0]=subset_extents[0]
gt_subset[3]=subset_extents[2]

In [ ]:
gt_subset

In [ ]:
dst_ds=gdal.GetDriverByName('Gtiff').Create('subset.tif',columns,rows,1)

In [ ]:
dst_ds.SetProjection(ds.GetProjection())
dst_ds.SetGeoTransform(gt_subset)

In [ ]:
dst_ds.GetRasterBand(1).WriteArray(data)

In [ ]:
dst_ds.FlushCache()

In [ ]:
dst_ds=None

### Resampling Raster

>Change the spatial resolution of raster dataset and set rules for aggregating or interpolating values across the new pixels sizes.

>Resampling techniques :

>>Nearest Neighbour

>>Bilinear

>>Cubic

>>etc.

In [ ]:
ds=gdal.Open(file_name)

In [ ]:
bands=ds.GetRasterBand(1)

In [ ]:
org_data=band.ReadAsArray()

In [ ]:
org_data.shape

In [ ]:
fine_data=band.ReadAsArray(buf_xsize=1151*2,buf_ysize=1151*2,resample_alg=gdal.GRA_Bilinear)

In [ ]:
coarse_data=band.ReadAsArray(buf_xsize=50,buf_ysize=50,resample_alg=gdal.GRA.NearestNeighbour)

In [ ]:
plt.figure(figsize=(20,25))
plt.subplot(1,3,1)
plt.imshow(org_data,cmap='gray')
plt.title('original')

plt.subplot(1,3,2)
plt.imshow(fine_data,cmap='gray')
plt.title('fine resample')

plt.subplot(1,3,3)
plt.imshow(coarse_data,cmap='gray')
plt.title('coarse resample')

In [ ]:
plt.close()